# Kaggle Competition: House Prices 
## Author: Justine Schabel 

### Data Exploration

First we need to import the required libraries. 

In [73]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Libraries for reading, cleaning and plotting the dataa
import numpy as np 
import pandas as pd 
import csv
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for models 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import LinearRegression

In [74]:
# Force numpy not to truncate output 
np.set_printoptions(threshold=100)

In [75]:
# Force pandas not to truncate output 
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

Read the data from the csv data files we downloaded from Kaggle

In [76]:
# Read in training data 
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

Split the training data into 80% training data and 20% dev data. We have 1168 training examples, 292 dev examples, 1495 test examples and 81 features (including the label).

In [77]:
# Split training data (labeled) into 80% training and 20% dev) and randomly sample 
training_data = train_df.sample(frac=0.8)
dev_data_df = train_df.drop(training_data.index)

# Examine shape of data sets
print("Training Set Size: ", training_data.shape)
print("Dev Set Size: ", dev_data_df.shape)
print("Test Set Size: ", test_df.shape)

Training Set Size:  (1168, 81)
Dev Set Size:  (292, 81)
Test Set Size:  (1459, 80)


We can also breifly look at a summary of the features. 
- The count attribute shows us that there are columns with missing data - we will need to manage NAs and look for outliers (ex. 9999 for NA)
- The mean attribute shows us that the average value of the features varies widely - we might consider standardizing 

In [78]:
# Briefly examine feature attributes for the training data 
training_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1168.000000,1168.000000,957.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1164.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1106.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000
mean,717.223459,56.536815,70.018809,10391.172945,6.117295,5.571918,1971.820205,1984.855308,108.680412,447.668664,45.881849,575.035103,1068.585616,1164.732021,350.286815,5.553082,1520.571918,0.422945,0.058219,1.571918,0.383562,2.872432,1.045377,6.524829,0.612158,1978.569620,1.769692,474.419521,94.063356,46.685788,21.769692,3.221747,15.014555,3.448630,41.455479,6.285959,2007.800514,181795.089041
std,425.252744,41.758850,23.235957,8964.925167,1.386349,1.112351,29.873487,20.646628,187.238696,461.266089,162.373404,447.886312,442.669834,388.366058,442.714385,48.240194,542.931085,0.514623,0.241463,0.550733,0.500355,0.828991,0.216292,1.653663,0.640872,24.828118,0.749205,214.062958,125.072960,66.691887,60.327960,29.184042,56.371615,44.896939,532.819316,2.669529,1.340337,80586.490922
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1880.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,346.750000,20.000000,60.000000,7500.000000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,218.750000,801.500000,882.000000,0.000000,0.000000,1122.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1962.000000,1.000000,327.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129800.000000
50%,713.500000,50.000000,70.000000,9458.500000,6.000000,5.000000,1974.000000,1993.000000,0.000000,391.000000,0.000000,486.500000,996.000000,1088.500000,0.000000,0.000000,1465.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,475.500000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,164995.000000
75%,1083.250000,70.000000,80.000000,11401.000000,7.000000,6.000000,2000.250000,2004.000000,170.250000,709.000000,0.000000,816.500000,1316.000000,1408.000000,729.250000,0.000000,1788.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.250000,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,215050.000000
max,1459.000000,190.000000,313.000000,164660.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,2.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


I will also consolodate our findings from the data_description file. Most features are either alphabetic (categorical) or numeric.

- Categorical Features:
    - Numeric:
        - MSSubCass: Identifies the type of dwelling involved in the sale
        - OverallQual: Rates the overall material and finish of the house (Likert Scale)
        - OverallCond: Rates the overall condition of the house (Likert Scale)
    - Alphabetic:   
        - MSZoning: Identifies the general zoning classification of the sale.
        - Street: Type of road access to property
        - Alley: Type of alley access to property
        - LotShape: General shape of property
        - LandContour: Flatness of the property
        - Utilities: Type of utilities available
        - LotConfig: Lot configuration
        - LandSlope: Slope of property
        - Neighborhood: Physical locations within Ames city limits
        - Condition1: Proximity to various conditions
        - Condition2: Proximity to various conditions (if more than one is present)
        - BldgType: Type of dwelling
        - HouseStyle: Style of dwelling
        - RoofStyle: Type of roof
        - RoofMatl: Roof material
        - Exterior1st: Exterior covering on house
        - Exterior2nd: Exterior covering on house (if more than one material)
        - MasVnrType: Masonry veneer type
        - ExterQual: Evaluates the quality of the material on the exterior 
        - ExterCond: Evaluates the present condition of the material on the exterior
        - Foundation: Type of foundation
        - BsmtQual: Evaluates the height of the basement
        - BsmtCond: Evaluates the general condition of the basement
        - BsmtExposure: Refers to walkout or garden level walls
        - BsmtFinType1: Rating of basement finished area
        - BsmtFinType2: Rating of basement finished area (if multiple types)
        - Heating: Type of heating
        - HeatingQC: Heating quality and condition
        - CentralAir: Central air conditioning (Y/N - This could be changed to 0/1 bool)
        - Electrical: Electrical system
        - KitchenQual: Kitchen quality
        - Functional: Home functionality (Assume typical unless deductions are warranted)
        - FireplaceQu: Fireplace quality
        - GarageType: Garage location
        - GarageFinish: Interior finish of the garage
        - GarageQual: Garage quality
        - GarageCond: Garage condition
        - PavedDrive: Paved driveway
        - PoolQC: Pool quality
        - Fence: Fence quality
        - MiscFeature: Miscellaneous feature not covered in other categories
        - SaleType: Type of sale
        - SaleCondition: Condition of sale    
- Numerical Features:
    - LotFrontage: Linear feet of street connected to property
    - LotArea: Lot size in square feet
    - YearBuilt: Original construction date
    - YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)
    - MasVnrArea: Masonry veneer area in square feet
    - BsmtFinSF1: Type 1 finished square feet
    - BsmtFinSF2: Type 2 finished square feet
    - BsmtUnfSF: Unfinished square feet of basement area
    - TotalBsmtSF: Total square feet of basement area
    - 1stFlrSF: First Floor square feet
    - 2ndFlrSF: Second floor square feet
    - LowQualFinSF: Low quality finished square feet (all floors)
    - GrLivArea: Above grade (ground) living area square feet
    - BsmtFullBath: Basement full bathrooms
    - BsmtHalfBath: Basement half bathrooms
    - FullBath: Full bathrooms above grade
    - HalfBath: Half baths above grade
    - Bedroom: Bedrooms above grade (does NOT include basement bedrooms)
    - Kitchen: Kitchens above grade
    - TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
    - Fireplaces: Number of fireplaces
    - GarageYrBlt: Year garage was built
    - GarageCars: Size of garage in car capacity
    - GarageArea: Size of garage in square feet
    - WoodDeckSF: Wood deck area in square feet
    - OpenPorchSF: Open porch area in square feet
    - EnclosedPorch: Enclosed porch area in square feet
    - 3SsnPorch: Three season porch area in square feet
    - ScreenPorch: Screen porch area in square feet
    - PoolArea: Pool area in square feet
    - MiscVal: Value of miscellaneous feature
    - MoSold: Month Sold (MM)
    - YrSold: Year Sold (YYYY)

I would like to examine all of the different types (specifically what numeric types there are).

In [79]:
training_data.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object


In order to do some distribution analysis, I will filter out the non-numeric features 

In [80]:
numeric_training_data = training_data.select_dtypes(include=['int64','float64'])
numeric_dev_data = dev_data_df.select_dtypes(include=['int64','float64'])
print("Numeric Training Data Shape: ", numeric_training_data.shape)
print("Numeric Dev Data Shape: ", numeric_dev_data.shape)

Numeric Training Data Shape:  (1168, 38)
Numeric Dev Data Shape:  (292, 38)


It appears that the only (numeric) features with missing data are LotFrontage, Masonry veneer area in square feet (MasVnrArea), Garage Year Built (GarageYrBlt). Since lot frontage is the length of the property that is connected to the street, I will drop it. I believe that this value would hold similar information as the lot area. I'll also drop the Id (as it has no real numerical meaning), as well as the numeric categorical data.

In [81]:
numeric_training_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1168.000000,1168.000000,957.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1164.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1106.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000
mean,717.223459,56.536815,70.018809,10391.172945,6.117295,5.571918,1971.820205,1984.855308,108.680412,447.668664,45.881849,575.035103,1068.585616,1164.732021,350.286815,5.553082,1520.571918,0.422945,0.058219,1.571918,0.383562,2.872432,1.045377,6.524829,0.612158,1978.569620,1.769692,474.419521,94.063356,46.685788,21.769692,3.221747,15.014555,3.448630,41.455479,6.285959,2007.800514,181795.089041
std,425.252744,41.758850,23.235957,8964.925167,1.386349,1.112351,29.873487,20.646628,187.238696,461.266089,162.373404,447.886312,442.669834,388.366058,442.714385,48.240194,542.931085,0.514623,0.241463,0.550733,0.500355,0.828991,0.216292,1.653663,0.640872,24.828118,0.749205,214.062958,125.072960,66.691887,60.327960,29.184042,56.371615,44.896939,532.819316,2.669529,1.340337,80586.490922
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1880.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,346.750000,20.000000,60.000000,7500.000000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,218.750000,801.500000,882.000000,0.000000,0.000000,1122.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1962.000000,1.000000,327.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129800.000000
50%,713.500000,50.000000,70.000000,9458.500000,6.000000,5.000000,1974.000000,1993.000000,0.000000,391.000000,0.000000,486.500000,996.000000,1088.500000,0.000000,0.000000,1465.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,475.500000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,164995.000000
75%,1083.250000,70.000000,80.000000,11401.000000,7.000000,6.000000,2000.250000,2004.000000,170.250000,709.000000,0.000000,816.500000,1316.000000,1408.000000,729.250000,0.000000,1788.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.250000,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,215050.000000
max,1459.000000,190.000000,313.000000,164660.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,2.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [82]:
numeric_dev_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,292.000000,292.000000,244.000000,292.000000,292.000000,292.000000,292.000000,292.000000,288.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.00000,292.000000,292.000000,273.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.0,292.000000,292.000000,292.000000,292.000000
mean,783.606164,58.339041,70.172131,11019.448630,6.027397,5.589041,1969.058219,1984.907534,83.496528,427.523973,49.219178,536.061644,1012.804795,1154.205479,333.815068,7.010274,1495.030822,0.434932,0.054795,1.537671,0.380137,2.842466,1.05137,6.489726,0.616438,1978.249084,1.756849,467.222603,94.969178,46.558219,22.691781,4.160959,15.246575,0.0,51.623288,6.465753,2007.876712,177425.623288
std,403.059560,44.446556,28.075499,13299.350173,1.369502,1.116396,31.441869,20.675907,152.239130,435.203341,157.276891,416.196891,420.253597,379.931617,411.317680,50.193200,449.204332,0.536514,0.227969,0.551741,0.513745,0.761517,0.23616,1.509386,0.660732,24.164043,0.740893,213.037842,126.610026,64.595277,64.288767,29.883476,53.322988,0.0,308.995416,2.836106,1.278371,74721.484916
min,12.000000,20.000000,21.000000,1491.000000,2.000000,2.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,372.000000,0.000000,0.000000,438.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,3.000000,0.000000,1910.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,2006.000000,55000.000000
25%,453.500000,20.000000,58.750000,7793.750000,5.000000,5.000000,1953.750000,1966.000000,0.000000,0.000000,0.000000,240.500000,768.000000,889.500000,0.000000,0.000000,1154.250000,0.000000,0.000000,1.000000,0.000000,2.000000,1.00000,6.000000,0.000000,1959.000000,1.000000,336.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,5.000000,2007.000000,131750.000000
50%,806.500000,50.000000,67.500000,9600.000000,6.000000,5.000000,1970.000000,1994.000000,0.000000,338.000000,0.000000,425.500000,977.000000,1079.000000,0.000000,0.000000,1460.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.00000,6.000000,1.000000,1978.000000,2.000000,484.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.0,0.000000,6.000000,2008.000000,159500.000000
75%,1128.750000,70.000000,80.000000,12191.500000,7.000000,6.000000,2000.000000,2004.000000,117.750000,723.000000,0.000000,751.500000,1202.500000,1337.500000,720.000000,0.000000,1742.250000,1.000000,0.000000,2.000000,1.000000,3.000000,1.00000,7.000000,1.000000,2002.000000,2.000000,572.000000,168.000000,66.000000,0.000000,0.000000,0.000000,0.0,0.000000,8.000000,2009.000000,200918.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2009.000000,2009.000000,894.000000,1721.000000,906.000000,2046.000000,3200.000000,3228.000000,1349.000000,513.000000,3228.000000,3.000000,1.000000,3.000000,2.000000,6.000000,3.00000,12.000000,3.000000,2009.000000,4.000000,1220.000000,736.000000,502.000000,386.000000,304.000000,288.000000,0.0,3500.000000,12.000000,2010.000000,451950.000000


### Feature Engineering

In [83]:
numeric_training_data = numeric_training_data.drop(columns=['Id','MasVnrArea', 'LotFrontage', 'MSSubClass', 'OverallQual', 'OverallCond', 'GarageYrBlt'])
numeric_dev_data = numeric_dev_data.drop(columns=['Id','MasVnrArea', 'LotFrontage', 'MSSubClass', 'OverallQual', 'OverallCond', 'GarageYrBlt'])

I also want to remove any columns that might be linear combinations. (TODO)
- BsmtUnfSF: Unfinished square feet of basement area and TotalBsmtSF: Total square feet of basement area
- I'll also remove MiscVal because there's not much interpretability 
- Consider combining deck/porch 
- Consider combining square feet of floors 
- Consider combing features in general (number of bathrooms, kitchens, floors etc) 

In [84]:
# Split into data and labels
train_data = numeric_training_data.drop(columns=["SalePrice"])
train_labels = numeric_training_data["SalePrice"]
dev_data = numeric_dev_data.drop(columns=["SalePrice"])
dev_labels = numeric_dev_data["SalePrice"]
test_data = test_df
# Double check the shape
print("Training Data Shape: ", train_data.shape)
print("Training Data Shape: ", dev_data.shape)
print("Training Labels Shape: ", train_labels.shape)
print("Training Labels Shape: ", dev_labels.shape)
print("Test Data Shape: ", test_data.shape)

Training Data Shape:  (1168, 30)
Training Data Shape:  (292, 30)
Training Labels Shape:  (1168,)
Training Labels Shape:  (292,)
Test Data Shape:  (1459, 80)


In [85]:
# Scale the Data
numeric_columns = ['LotArea', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold']
scaler = StandardScaler()
norm = scaler.fit(train_data[numeric_columns])
train_data[numeric_columns] = norm.transform(train_data[numeric_columns])
print(train_data.shape)
# Normalize features using the standard scaler [dev data]
dev_data[numeric_columns] = norm.transform(dev_data[numeric_columns])
print(dev_data.shape)
train_data.describe()

(1168, 30)
(292, 30)


,LotArea,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03,1.168000e+03
mean,8.246917e-17,3.725432e-15,-2.905781e-15,9.666925e-17,-4.065907e-16,8.564306e-17,2.614442e-16,-6.002618e-17,-1.028477e-16,2.080480e-16,-1.625412e-16,-9.144131e-17,4.861978e-17,7.338118e-17,-8.193598e-17,-6.313918e-17,1.403463e-16,-1.084559e-16,-3.878176e-17,-1.090737e-16,9.644350e-17,2.678603e-16,2.501804e-16,4.182347e-17,4.959408e-17,2.672900e-16,2.956159e-17,6.976915e-17,7.585257e-17,3.945179e-14
std,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00,1.000428e+00
min,-1.014517e+00,-3.074952e+00,-1.688907e+00,-9.709372e-01,-2.826910e-01,-1.284436e+00,-2.414990e+00,-2.139960e+00,-7.915642e-01,-1.151625e-01,-2.186429e+00,-8.222059e-01,-2.412136e-01,-2.855452e+00,-7.669074e-01,-3.466459e+00,-4.835236e+00,-2.737418e+00,-9.556031e-01,-2.363104e+00,-2.217211e+00,-7.523900e-01,-7.003219e-01,-3.610103e-01,-1.104414e-01,-2.664637e-01,-7.684505e-02,-7.783734e-02,-1.980958e+00,-1.343904e+00
25%,-3.226364e-01,-5.967780e-01,-8.651755e-01,-9.709372e-01,-2.826910e-01,-7.958219e-01,-6.036102e-01,-7.283158e-01,-7.915642e-01,-1.151625e-01,-7.335046e-01,-8.222059e-01,-2.412136e-01,-1.038912e+00,-7.669074e-01,-1.052853e+00,-2.098833e-01,-9.224866e-01,-9.556031e-01,-1.027784e+00,-6.854635e-01,-7.523900e-01,-7.003219e-01,-3.610103e-01,-1.104414e-01,-2.664637e-01,-7.684505e-02,-7.783734e-02,-4.819239e-01,-5.975039e-01
50%,-1.040803e-01,7.299878e-02,3.946495e-01,-1.229072e-01,-2.826910e-01,-1.977578e-01,-1.640426e-01,-1.963732e-01,-7.915642e-01,-1.151625e-01,-1.023992e-01,-8.222059e-01,-2.412136e-01,7.776285e-01,-7.669074e-01,1.539500e-01,-2.098833e-01,-3.175094e-01,6.054381e-01,3.075350e-01,5.049647e-03,-7.523900e-01,-3.403038e-01,-3.610103e-01,-1.104414e-01,-2.664637e-01,-7.684505e-02,-7.783734e-02,-1.071655e-01,1.488967e-01
75%,1.126902e-01,9.520808e-01,9.276523e-01,5.667949e-01,-2.826910e-01,5.393519e-01,5.591535e-01,6.266567e-01,8.563659e-01,-1.151625e-01,4.927746e-01,1.121795e+00,-2.412136e-01,7.776285e-01,1.232530e+00,1.539500e-01,-2.098833e-01,2.874677e-01,6.054381e-01,3.075350e-01,4.759072e-01,5.914013e-01,3.497307e-01,-3.610103e-01,-1.104414e-01,-2.664637e-01,-7.684505e-02,-7.783734e-02,6.423513e-01,8.952972e-01
max,1.721542e+01,1.278597e+00,1.218381e+00,1.127019e+01,8.799039e+00,3.933407e+00,1.139353e+01,9.086219e+00,3.874841e+00,1.174725e+01,7.594322e+00,3.065796e+00,8.045184e+00,2.594169e+00,3.231967e+00,6.187964e+00,9.040822e+00,4.522308e+00,3.727521e+00,2.978174e+00,4.409846e+00,6.102546e+00,7.505089e+00,8.792895e+00,1.730379e+01,8.252107e+00,1.636784e+01,2.902516e+01,2.141385e+00,1.641698e+00


### Models

#### Random Forest

In [86]:
# Try a random forest 
def RandomForest(num_trees):
    model = RandomForestClassifier(num_trees,max_depth=8)
    model.fit(train_data, train_labels)
    predictions = model.predict(dev_data)
    score = model.score(dev_data, dev_labels)
    print("Random Forest Performance for {0} trees: {1}".format(num_trees,score))
    # Plot_confusion_matrix
#     plot_confusion_matrix(model, dev_data, dev_labels, values_format = "d")
#     plt.title("{} Tree Random Forest Confusion Matrix:".format(num_trees))
#     plt.plot()
    
num_trees_list = [1,3,5,10,100]
for num_trees in num_trees_list:
    RandomForest(num_trees)

Random Forest Performance for 1 trees: 0.010273972602739725
Random Forest Performance for 3 trees: 0.023972602739726026
Random Forest Performance for 5 trees: 0.00684931506849315
Random Forest Performance for 10 trees: 0.010273972602739725
Random Forest Performance for 100 trees: 0.02054794520547945


#### Linear Regression

In [87]:
def LinearRegressionModel():
    model = LinearRegression()
    model.fit(train_data, train_labels)
    dev_predicted = model.predict(dev_data)
    score = model.score(dev_data, dev_labels)
    print("Mean Accuracy: ", score)
    
LinearRegressionModel()

Mean Accuracy:  0.8235644398819698


#### Logistic Regression

In [88]:
def LogRegressionModel():
#     model = LogisticRegression(penalty='l2', C=0.5, solver="liblinear", multi_class="auto")
    model = LogisticRegression()
    model.fit(train_data, train_labels)
    dev_predicted = model.predict(dev_data)
    score = model.score(dev_data, dev_labels)
    f1_score = metrics.f1_score(dev_labels, dev_predicted, average='weighted')
    print("F1 Score: {0} Mean Accuracy: {1}".format(f1_score, score))

LogRegressionModel()

F1 Score: 0.02470645792563601 Mean Accuracy: 0.023972602739726026


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### Multi-Layer Perceptron

In [89]:
def MLP():
    # Default activation is 'relu', random state lets us get the same result every time (so we can tune other parameters)
    # max_iter is 200 by default, but more helps. alpha is the regularization parameter. solver is 'adam' by default
    model = MLPClassifier(alpha=1e-3, hidden_layer_sizes=(100,), random_state=0, max_iter=300) 
    model.fit(train_data, train_labels) 
    score = model.score(dev_data, dev_labels)
    print("MLP accuracy = ",score)
MLP()

MLP accuracy =  0.0136986301369863


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


#### K-Nearest Neighbors

In [90]:
# Try K Nearest Neighbors - before any data cleaning 
def KNN(kn):
    model = KNeighborsClassifier(n_neighbors = kn)
    model.fit(train_data, train_labels)
    predictions = model.predict(dev_data)
    score = model.score(dev_data, dev_labels)
    print("KNN {0} neighbors : accuracy = {1}".format(kn,score))
    
# The alpha isn't actually making a difference 
neigh_list = [1,2,4, 7, 10]
for neigh in neigh_list:
    KNN(neigh)

KNN 1 neighbors : accuracy = 0.0
KNN 2 neighbors : accuracy = 0.003424657534246575
KNN 4 neighbors : accuracy = 0.00684931506849315
KNN 7 neighbors : accuracy = 0.010273972602739725
KNN 10 neighbors : accuracy = 0.010273972602739725
